# Importing Dependencies

In [1]:
import os
import json
import time

from dotenv import load_dotenv
from google import genai
from google.genai import types

In [2]:
load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [3]:
client = genai.Client(api_key=gemini_api_key)

# Load Results

In [4]:
model = 'gemini'
prompt_technique = 'base'

In [5]:
with open(f"../data/{model}/{prompt_technique}_prompt/first_level_subfeatures.json", "r", encoding="utf-8") as file:
    first_level_subfeatures = json.load(file)

with open(f"../data/{model}/{prompt_technique}_prompt/second_level_subfeatures.json", "r", encoding="utf-8") as file:
    second_level_subfeatures = json.load(file)

# Evalution Criteria

In [6]:
with open(f"../data/eval_criteria.json", "r", encoding="utf-8") as file:
    eval_criteria = json.load(file)

In [8]:
eval_criteria["super_feature_relation"][0]["rating"]

'Sub-feature'

In [9]:
eval_criteria["super_feature_relation"][0]["description"]

'A feature that is subordinated to the root feature.'

# Functions

In [ ]:
system_prompt = """
You are an expert in mobile app development and requirements engineering. 
Your primary role is to evaluate the refinement of software features, ensuring that the breakdown of high-level features into sub-features is logical, clear, relevant, and technically feasible.

You will assess the decomposition based on the following criteria:

1. **Feature Relationship Classification**:
   - Classify each sub-feature in relation to the root feature using one of the following:
     - **Sub-feature**: A feature that is directly derived from and dependent on the root feature.
     - **Sibling feature**: A feature at the same hierarchical level as the root feature.
     - **Super feature**: A feature that is broader or encompasses the root feature.
     - **Identical feature**: A feature that is essentially the same as the root feature.
     - **Other**: A feature that does not fit into the above categories.

2. **Relevance (Rating: 1 to 5)**:
   - Evaluate how relevant the sub-feature is to the root feature:
     - 5: Highly relevant and a natural extension.
     - 4: Mostly relevant and logically connected.
     - 3: Moderately relevant but might not serve the same purpose.
     - 2: Somewhat relevant, mainly because it belongs to the same app category.
     - 1: Not relevant at all.

3. **Clarity (Rating: 1 to 5)**:
   - Assess the clarity and understandability of the sub-feature description:
     - 5: Very clear and easily understandable.
     - 4: Mostly clear with minor syntax issues.
     - 3: Somewhat clear but contains ambiguities or is too lengthy.
     - 2: Mostly unclear and difficult to understand.
     - 1: Very unclear or irrelevant.

4. **Feasibility (Rating: 1 to 5)**:
   - Evaluate how practical and implementable the sub-feature is:
     - 5: Feasible and commonly implemented in existing apps.
     - 4: Feasible but lacks clear real-world examples.
     - 3: Probably feasible but has some uncertainties.
     - 2: Probably not feasible due to technical limitations.
     - 1: Not feasible at all.

**Instructions**:
- Provide a structured evaluation for each sub-feature.
- Use the above criteria to classify and rate the sub-feature.
"""

In [8]:
def get_prompt(feature: str, feature_description: str, super_feature: str, super_feature_description: str,) -> str:
    return f"""
Given the following mobile app feature and its refined sub-feature, evaluate the sub-feature based on the provided system guidelines.

**Feature**
```
feature: {super_feature}
description: {super_feature_description}
```

**Sub-feature to Evaluate**
```
sub_feature: {feature}
description: {feature_description}
```

Provide a structured evaluation of this sub-feature, based on provided criteria.

Return the evaluation as a structured JSON object in the following format:
{{
    "relationship": "Sub-feature | Sibling feature | Super feature | Identical feature | Other",
    "relevance": Rating (1-5),
    "clarity": Rating (1-5),
    "feasibility": Rating (1-5)
}}
"""

# Level 1 Feature Evaluation

In [ ]:
# # Para teste sobre menos sub-fetures
# first_level_subfeatures = [first_level_subfeatures[10]]
# first_level_subfeatures

[{'feature': 'Driver Guardian',
  'description': "Integrates with wearable devices to monitor the driver's vital signs, fatigue, and stress levels, providing recommendations for breaks or even assuming control of the vehicle if abnormal health patterns are detected.",
  'sub_features': [{'sub-feature': 'Wearable Device Integration & Pairing',
    'description': 'Enables secure and reliable connection with compatible wearable devices (e.g., smartwatches, fitness trackers) via Bluetooth or other protocols.  Handles device discovery, pairing, and data stream configuration. Includes support for managing multiple paired devices and user authentication for secure data transmission.'},
   {'sub-feature': 'Real-time Vital Sign Monitoring & Analysis',
    'description': 'Collects and analyzes real-time data from the wearable device, including heart rate, heart rate variability (HRV), sleep patterns, and potentially other relevant biometrics (e.g., skin temperature, galvanic skin response).  App

In [10]:
first_level_eval = []

for super_feature in first_level_subfeatures:
    for feature in super_feature["sub_features"]:
        prompt = get_prompt(
            feature["sub-feature"], 
            feature["description"],
            super_feature["feature"],
            super_feature["description"]
        )

        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(system_instruction=system_prompt),
            contents=prompt
        )

        first_level_eval.append({
            "super_feature": super_feature["feature"],
            "super_feature_description": super_feature["description"],
            "feature": feature["sub-feature"],
            "description": feature["description"],
            "feature_eval": json.loads((response.text).replace('```', '').replace('json', ''))
        })

        time.sleep(5)

# Salvar os resultados em um arquivo JSON
with open("../data/gemini/base_prompt/first_level_eval.json", "w") as f:
    json.dump(first_level_eval, f, indent=4)

# Level 2 Feature Evaluation

In [ ]:
# # Para teste sobre menos sub-fetures
# second_level_subfeatures = [second_level_subfeatures[55]]
# second_level_subfeatures

[{'super_feature': 'Laugh evaluation',
  'super_feature_description': 'The feature continually tracks the laughs of a user to count its quantity and assesses its authenticity, emotional context, and overall impact on social interactions.',
  'feature': 'Laugh Detection & Counting',
  'description': 'Utilizes microphone input and audio processing algorithms to detect and count instances of laughter from the user. Differentiates laughter from other sounds based on frequency, rhythm, and intensity.',
  'siblings': ['Laugh Detection & Counting',
   'Authenticity Assessment',
   'Emotional Context Inference',
   'Social Interaction Impact Analysis',
   'Laughter Data Visualization & Insights'],
  'sub_features': [{'sub-feature': 'Microphone Input Acquisition',
    'description': "Captures audio data from the user's microphone as the primary input source for laugh detection."},
   {'sub-feature': 'Audio Preprocessing',
    'description': 'Filters and cleans the audio data to remove noise and

In [12]:
second_level_eval = []

for super_feature in second_level_subfeatures:
    for feature in super_feature["sub_features"]:
        prompt = get_prompt(
            feature["sub-feature"], 
            feature["description"],
            super_feature["feature"],
            super_feature["description"]
        )

        response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(system_instruction=system_prompt),
            contents=prompt
        )

        second_level_eval.append({
            "root_feture": super_feature["super_feature"],
            "root_feature_description": super_feature["super_feature_description"],
            "super_feature": super_feature["feature"],
            "super_feature_description": super_feature["description"],
            "feature": feature["sub-feature"],
            "description": feature["description"],
            "feature_eval": json.loads((response.text).replace('```', '').replace('json', ''))
        })

        time.sleep(5)

# Salvar os resultados em um arquivo JSON
with open("../data/gemini/base_prompt/second_level_eval.json", "w") as f:
    json.dump(second_level_eval, f, indent=4)